In [4]:
# libraries
from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np 
import pandas as pd 

import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import nltk
from nltk.corpus import stopwords                 
from nltk.stem.wordnet import WordNetLemmatizer    
from nltk import word_tokenize, pos_tag           

# stopwords for preprocessing
nltk.download('stopwords') 
stopwords_english = stopwords.words('english') 

import seaborn as sns
import matplotlib.pyplot as plt
import os

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# data 
train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data/train.csv') 
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data/test.csv')
test_label = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data/test_labels.csv')

In [ ]:
# train.csv
train.head(10)
train.info()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [ ]:
# test.csv

test.head(10)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            153164 non-null  object
 1   comment_text  153164 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


In [ ]:
# test_labels.csv

test_label.head(10)
test_label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63978 entries, 5 to 153156
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             63978 non-null  object
 1   toxic          63978 non-null  int64 
 2   severe_toxic   63978 non-null  int64 
 3   obscene        63978 non-null  int64 
 4   threat         63978 non-null  int64 
 5   insult         63978 non-null  int64 
 6   identity_hate  63978 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 3.9+ MB


# Preprocessing Data

In [ ]:
# preprocess the comments : make text lowercase, remove hyperlinks, punctuation, digits, and stopwords.
def preprocess(comment):

    for word in comment:

        word = word.lower()                                               # lowercase
        word = re.sub(r'https?://[^\s\n\r]+', '', word)                   # remove links
        word = re.sub('[%s]' % re.escape(string.punctuation), '', word)   # remove punctuation
        word = re.sub(r'\d', '', word)                                    # remove digits
    
        yield ' '.join([s for s in word.split(' ') if word not in stopwords_english])

# list of clean comments
clean_comments = list(preprocess(train['comment_text']))